# 🔴 Connexion aux Bases de Données

**Badge:** 🔴 Avancé | ⏱ 60 min | 🔑 **Concepts clés :** DB API 2.0, sqlite3, psycopg2, SQLAlchemy

## Objectifs

- Comprendre l'interface DB API 2.0 standard Python
- Manipuler une base SQLite embarquée avec le module `sqlite3`
- Gérer les transactions (commit, rollback)
- Utiliser des requêtes paramétrées pour éviter les injections SQL
- Découvrir SQLAlchemy Core et ORM
- Stocker et requêter notre dataset e-commerce dans une base de données

## Prérequis

- Bases de SQL (SELECT, INSERT, UPDATE, DELETE)
- Pandas pour la manipulation de données
- Context managers (`with` statement)

## 1. DB API 2.0 : L'interface standard

**DB API 2.0** (PEP 249) est l'interface standard pour tous les connecteurs de bases de données en Python.

### Concepts clés :

- **Connection** : représente la connexion à la base de données
- **Cursor** : permet d'exécuter des requêtes et de récupérer les résultats
- **Paramètres** : protection contre les injections SQL
- **Transactions** : commit() et rollback()

Tous les connecteurs (sqlite3, psycopg2, mysql-connector, etc.) suivent cette interface.

## 2. SQLite : Base de données embarquée

SQLite est une base de données légère, sans serveur, parfaite pour :
- Apprendre le SQL
- Prototyper
- Stocker des données locales
- Applications mobiles/desktop

**Avantages** : zéro configuration, fichier unique, inclus dans Python  
**Limitations** : pas de concurrence écriture, pas de haute disponibilité

In [ ]:
import sqlite3
import pandas as pd
from pathlib import Path
from datetime import datetime

# Connexion à une base SQLite (crée le fichier si inexistant)
conn = sqlite3.connect('ecommerce.db')

# Créer un curseur pour exécuter des commandes
cursor = conn.cursor()

# Créer une table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS products (
        product_id INTEGER PRIMARY KEY,
        product_name TEXT NOT NULL,
        category TEXT,
        price REAL
    )
''')

# Commit pour valider les changements
conn.commit()

print("Table 'products' créée avec succès")

# Toujours fermer les connexions
cursor.close()
conn.close()

### Meilleure pratique : Context Manager

Utilisez `with` pour garantir la fermeture automatique des connexions :

In [ ]:
# Avec context manager - fermeture automatique
with sqlite3.connect('ecommerce.db') as conn:
    cursor = conn.cursor()
    
    # Insertion de données
    cursor.execute('''
        INSERT INTO products (product_name, category, price)
        VALUES ('Laptop', 'Electronics', 999.99)
    ''')
    
    conn.commit()
    print("Produit inséré")

# Connexion fermée automatiquement ici

## 3. Requêtes paramétrées : JAMAIS de f-strings !

### ❌ DANGER : Injection SQL

In [ ]:
# ❌ NE JAMAIS FAIRE ÇA - vulnérable aux injections SQL
category = "Electronics"
query = f"SELECT * FROM products WHERE category = '{category}'"

# Un attaquant pourrait passer : "Electronics'; DROP TABLE products; --"

print("❌ Évitez absolument cette pratique !")

### ✅ CORRECT : Paramètres avec placeholders

In [ ]:
# ✅ Utilisez des placeholders (? pour SQLite, %s pour PostgreSQL)
with sqlite3.connect('ecommerce.db') as conn:
    cursor = conn.cursor()
    
    # Paramètre unique
    category = "Electronics"
    cursor.execute(
        "SELECT * FROM products WHERE category = ?",
        (category,)  # Tuple, même pour un seul paramètre
    )
    results = cursor.fetchall()
    
    print(f"Produits trouvés : {len(results)}")
    
    # Paramètres multiples
    cursor.execute(
        "SELECT * FROM products WHERE category = ? AND price < ?",
        (category, 1500)
    )
    results = cursor.fetchall()
    print(f"Produits < 1500€ : {len(results)}")

## 4. Opérations CRUD (Create, Read, Update, Delete)

In [ ]:
with sqlite3.connect('ecommerce.db') as conn:
    cursor = conn.cursor()
    
    # CREATE - Insertion
    products = [
        ('Smartphone', 'Electronics', 699.99),
        ('Desk Chair', 'Furniture', 249.99),
        ('Coffee Maker', 'Appliances', 89.99),
        ('Headphones', 'Electronics', 149.99)
    ]
    
    cursor.executemany(
        "INSERT INTO products (product_name, category, price) VALUES (?, ?, ?)",
        products
    )
    
    conn.commit()
    print(f"✓ {cursor.rowcount} produits insérés")
    
    # READ - Lecture
    cursor.execute("SELECT * FROM products")
    all_products = cursor.fetchall()
    print(f"\n✓ Total produits : {len(all_products)}")
    
    # Affichage avec noms de colonnes
    columns = [desc[0] for desc in cursor.description]
    print(f"Colonnes : {columns}")
    print("\nPremiers produits :")
    for product in all_products[:3]:
        print(product)
    
    # UPDATE - Mise à jour
    cursor.execute(
        "UPDATE products SET price = price * 0.9 WHERE category = ?",
        ('Electronics',)
    )
    conn.commit()
    print(f"\n✓ {cursor.rowcount} produits mis à jour (réduction 10%)")
    
    # DELETE - Suppression
    cursor.execute(
        "DELETE FROM products WHERE price < ?",
        (100,)
    )
    conn.commit()
    print(f"✓ {cursor.rowcount} produits supprimés (prix < 100€)")

## 5. Méthodes de récupération de résultats

In [ ]:
with sqlite3.connect('ecommerce.db') as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM products ORDER BY price DESC")
    
    # fetchone() : récupère une seule ligne
    first_row = cursor.fetchone()
    print("fetchone() - Produit le plus cher :")
    print(first_row)
    
    # fetchmany(n) : récupère n lignes
    print("\nfetchmany(2) - 2 produits suivants :")
    next_rows = cursor.fetchmany(2)
    for row in next_rows:
        print(row)
    
    # fetchall() : récupère toutes les lignes restantes
    print("\nfetchall() - Tous les produits restants :")
    remaining_rows = cursor.fetchall()
    print(f"Lignes restantes : {len(remaining_rows)}")

## 6. Transactions : Commit et Rollback

In [ ]:
# Exemple de transaction avec rollback en cas d'erreur
with sqlite3.connect('ecommerce.db') as conn:
    cursor = conn.cursor()
    
    try:
        # Début de transaction
        cursor.execute(
            "INSERT INTO products (product_name, category, price) VALUES (?, ?, ?)",
            ('Tablet', 'Electronics', 399.99)
        )
        
        # Simulation d'une erreur
        # cursor.execute("INSERT INTO wrong_table VALUES (1)")  # Décommenter pour tester
        
        cursor.execute(
            "INSERT INTO products (product_name, category, price) VALUES (?, ?, ?)",
            ('Monitor', 'Electronics', 299.99)
        )
        
        # Si tout va bien, commit
        conn.commit()
        print("✓ Transaction réussie : 2 produits insérés")
        
    except sqlite3.Error as e:
        # En cas d'erreur, rollback
        conn.rollback()
        print(f"❌ Erreur : {e}")
        print("Transaction annulée (rollback)")

## 7. Integration avec Pandas : Charger un DataFrame

In [ ]:
# Créer un DataFrame de ventes e-commerce
df_sales = pd.DataFrame({
    'order_id': range(1, 11),
    'customer_name': ['Alice', 'Bob', 'Charlie', 'Diana', 'Eve', 
                      'Frank', 'Grace', 'Henry', 'Iris', 'Jack'],
    'product_name': ['Laptop', 'Smartphone', 'Headphones', 'Tablet', 'Monitor',
                     'Laptop', 'Smartphone', 'Headphones', 'Tablet', 'Monitor'],
    'quantity': [1, 2, 1, 1, 2, 1, 1, 3, 1, 1],
    'total_amount': [999.99, 1399.98, 149.99, 399.99, 599.98,
                     999.99, 699.99, 449.97, 399.99, 299.99],
    'order_date': pd.date_range('2024-01-01', periods=10, freq='D')
})

print("DataFrame créé :")
print(df_sales.head())

# Charger dans SQLite avec Pandas
with sqlite3.connect('ecommerce.db') as conn:
    # to_sql() : charge un DataFrame dans une table
    df_sales.to_sql(
        name='sales',           # Nom de la table
        con=conn,               # Connexion
        if_exists='replace',    # 'fail', 'replace', 'append'
        index=False             # Ne pas sauvegarder l'index
    )
    
    print("\n✓ DataFrame chargé dans la table 'sales'")
    
    # Vérification
    cursor = conn.cursor()
    cursor.execute("SELECT COUNT(*) FROM sales")
    count = cursor.fetchone()[0]
    print(f"Nombre de ventes dans la base : {count}")

## 8. Requêter et récupérer dans un DataFrame

In [ ]:
with sqlite3.connect('ecommerce.db') as conn:
    # read_sql_query() : exécute une requête SQL et retourne un DataFrame
    query = """
        SELECT 
            product_name,
            SUM(quantity) as total_quantity,
            SUM(total_amount) as total_revenue,
            COUNT(*) as order_count
        FROM sales
        GROUP BY product_name
        ORDER BY total_revenue DESC
    """
    
    df_results = pd.read_sql_query(query, conn)
    
    print("Analyse des ventes par produit :")
    print(df_results)
    
    # Requête paramétrée avec Pandas
    query_filtered = """
        SELECT * FROM sales 
        WHERE total_amount > ?
        ORDER BY order_date DESC
    """
    
    df_high_value = pd.read_sql_query(
        query_filtered, 
        conn, 
        params=(500,)
    )
    
    print(f"\nVentes > 500€ : {len(df_high_value)} commandes")

## 9. SQLAlchemy Core : Alternative plus puissante

In [ ]:
# Installation : pip install sqlalchemy
from sqlalchemy import create_engine, text

# Créer un engine (pool de connexions)
engine = create_engine('sqlite:///ecommerce.db', echo=False)

# Exécuter une requête avec SQLAlchemy
with engine.connect() as conn:
    # text() : requête SQL paramétrée
    result = conn.execute(
        text("SELECT * FROM sales WHERE total_amount > :amount"),
        {"amount": 600}
    )
    
    print("Ventes > 600€ avec SQLAlchemy :")
    for row in result:
        print(f"Order {row.order_id}: {row.customer_name} - {row.total_amount}€")

# Integration Pandas + SQLAlchemy
df_sqla = pd.read_sql_query(
    text("SELECT product_name, AVG(total_amount) as avg_amount FROM sales GROUP BY product_name"),
    engine
)

print("\nMontant moyen par produit :")
print(df_sqla)

## 10. SQLAlchemy ORM : Vue d'ensemble

L'ORM (Object-Relational Mapping) permet de manipuler des bases de données via des classes Python.

In [ ]:
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column, Session
from sqlalchemy import String, Float, Integer

# Définir la base
class Base(DeclarativeBase):
    pass

# Définir un modèle (= table)
class Product(Base):
    __tablename__ = 'products_orm'
    
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(100))
    category: Mapped[str] = mapped_column(String(50))
    price: Mapped[float] = mapped_column(Float)
    
    def __repr__(self):
        return f"<Product(name='{self.name}', price={self.price})>"

# Créer les tables
engine_orm = create_engine('sqlite:///ecommerce_orm.db')
Base.metadata.create_all(engine_orm)

# Ajouter des données
with Session(engine_orm) as session:
    # Créer des objets Python
    prod1 = Product(name='Laptop Pro', category='Electronics', price=1299.99)
    prod2 = Product(name='Wireless Mouse', category='Accessories', price=29.99)
    
    # Ajouter à la session
    session.add_all([prod1, prod2])
    session.commit()
    
    print("✓ Produits ajoutés avec ORM")
    
    # Requêter
    products = session.query(Product).filter(Product.category == 'Electronics').all()
    print(f"\nProduits Electronics : {products}")

print("\n💡 L'ORM sera approfondi dans les modules avancés")

## 11. psycopg2 : Connecteur PostgreSQL

PostgreSQL est une base de données relationnelle robuste, idéale pour la production.

```python
# Installation : pip install psycopg2-binary
import psycopg2

# Connexion à PostgreSQL
conn = psycopg2.connect(
    host="localhost",
    database="ecommerce",
    user="postgres",
    password="password"
)

# Même interface DB API 2.0
cursor = conn.cursor()
cursor.execute("SELECT version()")
version = cursor.fetchone()
print(f"PostgreSQL version : {version}")

# Attention : placeholders sont %s (pas ?)
cursor.execute(
    "SELECT * FROM products WHERE category = %s",
    ('Electronics',)
)
```

**Note** : Installation non requise pour ce notebook (mention pour culture générale).

## 12. Cas pratique : Pipeline complet CSV → SQLite → Analyse

In [ ]:
# 1. Créer un dataset e-commerce plus complet
import numpy as np

np.random.seed(42)
dates = pd.date_range('2024-01-01', '2024-12-31', freq='D')
products = ['Laptop', 'Smartphone', 'Tablet', 'Headphones', 'Monitor', 'Keyboard']
categories = {'Laptop': 'Electronics', 'Smartphone': 'Electronics', 
              'Tablet': 'Electronics', 'Headphones': 'Accessories',
              'Monitor': 'Electronics', 'Keyboard': 'Accessories'}
prices = {'Laptop': 999.99, 'Smartphone': 699.99, 'Tablet': 399.99,
          'Headphones': 149.99, 'Monitor': 299.99, 'Keyboard': 79.99}

data = []
for i, date in enumerate(dates[:100]):  # 100 ventes
    product = np.random.choice(products)
    quantity = np.random.randint(1, 5)
    data.append({
        'order_id': i + 1,
        'order_date': date,
        'product_name': product,
        'category': categories[product],
        'quantity': quantity,
        'unit_price': prices[product],
        'total_amount': round(prices[product] * quantity, 2)
    })

df_ecommerce = pd.DataFrame(data)
print(f"Dataset créé : {len(df_ecommerce)} ventes")
print(df_ecommerce.head())

# 2. Sauvegarder en CSV
csv_path = 'ecommerce_sales.csv'
df_ecommerce.to_csv(csv_path, index=False)
print(f"\n✓ Sauvegardé : {csv_path}")

In [ ]:
# 3. Charger le CSV dans SQLite
db_path = 'ecommerce_full.db'

with sqlite3.connect(db_path) as conn:
    # Lire CSV et charger dans la base
    df_from_csv = pd.read_csv(csv_path, parse_dates=['order_date'])
    
    df_from_csv.to_sql(
        name='orders',
        con=conn,
        if_exists='replace',
        index=False
    )
    
    print(f"✓ {len(df_from_csv)} commandes chargées dans SQLite")
    
    # 4. Analyses SQL
    analyses = {
        'Ventes par catégorie': """
            SELECT category, 
                   COUNT(*) as orders,
                   SUM(total_amount) as revenue
            FROM orders
            GROUP BY category
        """,
        'Top 5 produits': """
            SELECT product_name,
                   SUM(quantity) as total_qty,
                   SUM(total_amount) as revenue
            FROM orders
            GROUP BY product_name
            ORDER BY revenue DESC
            LIMIT 5
        """,
        'Ventes mensuelles': """
            SELECT strftime('%Y-%m', order_date) as month,
                   COUNT(*) as orders,
                   SUM(total_amount) as revenue
            FROM orders
            GROUP BY month
            ORDER BY month
        """
    }
    
    for name, query in analyses.items():
        print(f"\n{'='*50}")
        print(f"{name}")
        print('='*50)
        df_result = pd.read_sql_query(query, conn)
        print(df_result.to_string(index=False))

## Pièges courants

### 1. Injection SQL avec f-strings

In [ ]:
# ❌ DANGER
user_input = "Electronics"
# query = f"SELECT * FROM products WHERE category = '{user_input}'"

# ✅ CORRECT
with sqlite3.connect('ecommerce.db') as conn:
    cursor = conn.cursor()
    cursor.execute(
        "SELECT * FROM products WHERE category = ?",
        (user_input,)
    )
    print(f"✓ Requête sécurisée : {len(cursor.fetchall())} résultats")

### 2. Oublier commit()

In [ ]:
# ❌ Changements non sauvegardés
conn = sqlite3.connect('test.db')
cursor = conn.cursor()
cursor.execute("CREATE TABLE IF NOT EXISTS test (id INTEGER)")
cursor.execute("INSERT INTO test VALUES (1)")
# Pas de commit ! Les données sont perdues si le programme crash
conn.close()

# ✅ Toujours committer
conn = sqlite3.connect('test.db')
cursor = conn.cursor()
cursor.execute("INSERT INTO test VALUES (2)")
conn.commit()  # Sauvegarde garantie
conn.close()

print("✓ Données commitées")

### 3. Curseur ou connexion non fermés

In [ ]:
# ❌ Fuite de ressources
# conn = sqlite3.connect('db.db')
# cursor = conn.cursor()
# ... travail ...
# Oubli de fermer → fuite mémoire

# ✅ Context manager
with sqlite3.connect('test.db') as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM test")
    print(f"✓ {len(cursor.fetchall())} résultats")
# Fermeture automatique garantie

### 4. Confusion entre placeholders SQLite (?) et PostgreSQL (%s)

In [ ]:
# SQLite : utilise ?
with sqlite3.connect('test.db') as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM test WHERE id = ?", (1,))
    print("✓ SQLite : placeholder '?'")

# PostgreSQL : utilise %s
# cursor.execute("SELECT * FROM test WHERE id = %s", (1,))
print("✓ PostgreSQL : placeholder '%s'")

# SQLAlchemy : utilise :name (named parameters)
print("✓ SQLAlchemy : placeholder ':name'")

## Mini-exercices

### Exercice 1 : CRUD complet

Créez une table `customers` avec les champs : `id`, `name`, `email`, `country`.  
Insérez 5 clients, affichez tous les clients français, mettez à jour un email, supprimez un client.

In [ ]:
# Votre code ici


### Exercice 2 : Requêtes paramétrées

À partir de la table `orders` créée précédemment :  
1. Trouvez toutes les commandes d'un produit donné (paramètre)  
2. Trouvez les commandes entre deux dates (2 paramètres)  
3. Trouvez les commandes > montant X pour une catégorie Y (2 paramètres)

In [ ]:
# Votre code ici


### Exercice 3 : CSV → SQLite → Analyse

1. Créez un nouveau CSV avec 50 transactions (colonnes : transaction_id, date, customer_id, amount)  
2. Chargez-le dans une table SQLite `transactions`  
3. Calculez le montant total par client (GROUP BY)  
4. Récupérez le résultat dans un DataFrame et affichez un graphique

In [ ]:
# Votre code ici


## Solutions des exercices

In [ ]:
# Solution Exercice 1
with sqlite3.connect('customers.db') as conn:
    cursor = conn.cursor()
    
    # Créer table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS customers (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            email TEXT UNIQUE,
            country TEXT
        )
    ''')
    
    # Insérer 5 clients
    customers = [
        ('Alice Dupont', 'alice@email.fr', 'France'),
        ('Bob Smith', 'bob@email.com', 'USA'),
        ('Charlie Martin', 'charlie@email.fr', 'France'),
        ('Diana Johnson', 'diana@email.co.uk', 'UK'),
        ('Eve Bernard', 'eve@email.fr', 'France')
    ]
    
    cursor.executemany(
        "INSERT INTO customers (name, email, country) VALUES (?, ?, ?)",
        customers
    )
    conn.commit()
    print(f"✓ {cursor.rowcount} clients insérés")
    
    # Lire clients français
    cursor.execute("SELECT * FROM customers WHERE country = ?", ('France',))
    french_customers = cursor.fetchall()
    print(f"\nClients français : {len(french_customers)}")
    for customer in french_customers:
        print(customer)
    
    # Mettre à jour email
    cursor.execute(
        "UPDATE customers SET email = ? WHERE name = ?",
        ('alice.dupont@newemail.fr', 'Alice Dupont')
    )
    conn.commit()
    print(f"\n✓ {cursor.rowcount} email mis à jour")
    
    # Supprimer un client
    cursor.execute("DELETE FROM customers WHERE name = ?", ('Bob Smith',))
    conn.commit()
    print(f"✓ {cursor.rowcount} client supprimé")
    
    # Vérification finale
    cursor.execute("SELECT COUNT(*) FROM customers")
    print(f"\nTotal clients restants : {cursor.fetchone()[0]}")

In [ ]:
# Solution Exercice 2
with sqlite3.connect('ecommerce_full.db') as conn:
    # 1. Commandes d'un produit
    product = 'Laptop'
    df_product = pd.read_sql_query(
        "SELECT * FROM orders WHERE product_name = ?",
        conn,
        params=(product,)
    )
    print(f"Commandes de {product} : {len(df_product)}")
    
    # 2. Commandes entre deux dates
    df_dates = pd.read_sql_query(
        "SELECT * FROM orders WHERE order_date BETWEEN ? AND ?",
        conn,
        params=('2024-01-01', '2024-01-31')
    )
    print(f"\nCommandes en janvier : {len(df_dates)}")
    
    # 3. Commandes > montant pour catégorie
    df_filtered = pd.read_sql_query(
        "SELECT * FROM orders WHERE total_amount > ? AND category = ?",
        conn,
        params=(500, 'Electronics')
    )
    print(f"\nCommandes Electronics > 500€ : {len(df_filtered)}")
    print(df_filtered[['order_id', 'product_name', 'total_amount']].head())

In [ ]:
# Solution Exercice 3
import matplotlib.pyplot as plt

# 1. Créer CSV
np.random.seed(123)
df_transactions = pd.DataFrame({
    'transaction_id': range(1, 51),
    'date': pd.date_range('2024-01-01', periods=50, freq='D'),
    'customer_id': np.random.randint(1, 11, 50),
    'amount': np.random.uniform(10, 500, 50).round(2)
})

df_transactions.to_csv('transactions.csv', index=False)
print("✓ CSV créé")

# 2. Charger dans SQLite
with sqlite3.connect('transactions.db') as conn:
    df_transactions.to_sql('transactions', conn, if_exists='replace', index=False)
    print("✓ Chargé dans SQLite")
    
    # 3. Analyse GROUP BY
    query = """
        SELECT customer_id,
               COUNT(*) as transaction_count,
               SUM(amount) as total_amount,
               AVG(amount) as avg_amount
        FROM transactions
        GROUP BY customer_id
        ORDER BY total_amount DESC
    """
    
    df_customer_summary = pd.read_sql_query(query, conn)
    print("\nRésumé par client :")
    print(df_customer_summary)
    
    # 4. Graphique
    plt.figure(figsize=(10, 6))
    plt.bar(
        df_customer_summary['customer_id'].astype(str),
        df_customer_summary['total_amount']
    )
    plt.xlabel('Customer ID')
    plt.ylabel('Montant total (€)')
    plt.title('Montant total par client')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    print(f"\n✓ Top client : Customer {df_customer_summary.iloc[0]['customer_id']} "
          f"avec {df_customer_summary.iloc[0]['total_amount']:.2f}€")

## Résumé

### Points clés

1. **DB API 2.0** : interface standard pour tous les connecteurs Python
2. **sqlite3** : base embarquée parfaite pour apprendre et prototyper
3. **Requêtes paramétrées** : TOUJOURS utiliser des placeholders (?, %s, :name)
4. **Transactions** : commit() pour valider, rollback() pour annuler
5. **Context managers** : `with` garantit la fermeture des connexions
6. **Integration Pandas** : to_sql() et read_sql_query() facilitent les échanges
7. **SQLAlchemy** : alternative puissante avec ORM pour projets complexes

### Prochaines étapes

- Notebook suivant : **Concurrence et asyncio**
- Approfondir : PostgreSQL, ORM SQLAlchemy
- Outils avancés : Alembic (migrations), connection pooling